# Actionable Segmentation Engine - Quick Start Demo

This notebook demonstrates the core capabilities of the Actionable Segmentation Engine.

## Overview
The engine transforms customer event data into actionable business segments using:
- **ML Clustering**: KMeans with automatic k-selection
- **Robustness Testing**: Sensitivity analysis for segment stability
- **Actionability Evaluation**: LLM-powered assessment of commercial viability
- **Business Insights**: Human-readable explanations and recommendations

In [ ]:
# Setup: Add the parent directory to the path
import sys
sys.path.insert(0, '..')

## 1. Quick Segmentation (One-Liner)

The simplest way to run the pipeline is with `quick_segmentation()`:

In [ ]:
from src.pipeline import quick_segmentation, format_pipeline_summary

# Run with 500 synthetic customers, 5 segments
result = quick_segmentation(
    n_customers=500,
    n_clusters=5,
    seed=42,
    verbose=True
)

# Print summary
print(format_pipeline_summary(result))

## 2. Exploring the Results

The `PipelineResult` contains all outputs from every stage:

In [ ]:
# Overall summary
summary = result.get_summary()
print("Pipeline Summary:")
print(f"  Total customers profiled: {summary['total_customers']}")
print(f"  Total segments created: {summary['total_segments']}")
print(f"  Valid segments: {summary['valid_segments']}")
print(f"  Actionable segments: {summary['actionable_segments']}")
print(f"  Production-ready segments: {summary['production_ready_segments']}")

In [ ]:
# View all segments
print("\nSegment Overview:")
print("-" * 70)
for segment in result.segments:
    print(f"  {segment.name}")
    print(f"    - Size: {segment.size} customers")
    print(f"    - Total CLV: ${float(segment.total_clv):,.2f}")
    print(f"    - Avg CLV: ${float(segment.avg_clv):,.2f}")
    print(f"    - Traits: {', '.join(segment.defining_traits[:3])}")
    print()

## 3. Filtering Segments

Use built-in properties to filter segments by quality:

In [ ]:
# Get actionable segments only
actionable = result.actionable_segments
print(f"Actionable segments: {len(actionable)}")
for seg in actionable:
    eval_ = result.actionability_evaluations[seg.segment_id]
    print(f"  - {seg.name}: {eval_.recommended_action}")

In [ ]:
# Get production-ready segments (valid AND actionable)
production_ready = result.production_ready_segments
print(f"\nProduction-ready segments: {len(production_ready)}")
for seg in production_ready:
    rob = result.robustness_scores.get(seg.segment_id)
    if rob:
        print(f"  - {seg.name}: robustness={rob.overall_robustness:.2f}, tier={rob.robustness_tier.value}")

## 4. Detailed Segment Analysis

Get complete details for any segment:

In [ ]:
from src.pipeline import print_segment_details

# Show details for the largest segment
largest_segment = max(result.segments, key=lambda s: s.size)
print_segment_details(result, largest_segment.segment_id)

## 5. Business Explanations

View the LLM-generated explanations for each segment:

In [ ]:
print("Segment Business Explanations:")
print("=" * 70)
for segment in result.segments:
    explanation = result.explanations.get(segment.segment_id)
    if explanation:
        print(f"\n{segment.name}")
        print(f"Summary: {explanation.executive_summary}")
        print(f"Campaign: {explanation.recommended_campaign}")
        print(f"ROI: {explanation.expected_roi}")
        print(f"Confidence: {explanation.confidence_level.value}")

## 6. Visualizations

Create visualizations using the reporting module:

In [ ]:
from src.reporting import (
    set_style,
    plot_segment_distribution,
    plot_segment_sizes_pie,
    plot_robustness_scores,
    show_figure,
)

# Set visual style
set_style("whitegrid")

In [ ]:
# Segment size distribution
from src.reporting.segment_reporter import segment_to_summary

summaries = [segment_to_summary(seg) for seg in result.segments]

fig = plot_segment_distribution(summaries, by="size", title="Segment Size Distribution")
show_figure(fig)

In [ ]:
# Segment sizes pie chart
fig = plot_segment_sizes_pie(summaries, title="Segment Size Distribution (Pie)")
show_figure(fig)

In [ ]:
# CLV distribution
fig = plot_segment_distribution(summaries, by="clv", title="Segment Total CLV")
show_figure(fig)

In [ ]:
# Robustness scores
if result.robustness_scores:
    fig = plot_robustness_scores(result.robustness_scores, title="Segment Robustness Scores")
    show_figure(fig)

## 7. Export Results

Export results to JSON for further processing:

In [ ]:
import json
from src.pipeline import export_results_to_dict

# Export to dict
export_data = export_results_to_dict(result)

# Preview structure
print("Export structure:")
print(f"  success: {export_data['success']}")
print(f"  duration_ms: {export_data['duration_ms']:.1f}")
print(f"  n_profiles: {export_data['n_profiles']}")
print(f"  n_segments: {export_data['n_segments']}")

In [ ]:
# Save to file (optional)
# with open('segmentation_results.json', 'w') as f:
#     json.dump(export_data, f, indent=2)

## Next Steps

- **02_custom_configuration.ipynb**: Learn about advanced configuration options
- **03_visualization_gallery.ipynb**: Explore all visualization types
- **04_working_with_real_data.ipynb**: Connect to your own data sources